In [1]:
import os
os.chdir("../")
%pwd

'e:\\Deep Learning\\TENSORFLOW\\rice_image_detection'

In [2]:
import mlflow

In [3]:
MLFLOW_TRACKING_URI = "https://dagshub.com/karmakaragradwip02/rice_image_detection_cnn.mlflow"
os.environ['MLFLOW_TRACKING_URI'] = MLFLOW_TRACKING_URI
os.environ['MLFLOW_TRACKING_USERNAME'] = 'karmakaragradwip02'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '9ccb0f28354fcca6469017b32544fa0704b9c343'

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen= True)
class ModelEvaluationConfig:
    root_dir: Path
    trained_model_dir: Path
    val_dir: Path
    history_dir: Path
    graph_dir: Path
    mlflow_uri: str
    all_params: dict

In [5]:
import mlflow
import mlflow.keras
import tensorflow as tf

In [6]:
from src.RICE_IMAGE_DETECTION.constants import *
from src.RICE_IMAGE_DETECTION.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigureationManager:
    def __init__(self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = Path(config.root_dir),
            trained_model_dir = Path(config.trained_model_dir),
            history_dir= Path(config.history_dir),
            graph_dir = Path(config.graph_dir),
            val_dir= Path(config.val_dir),
            mlflow_uri="https://dagshub.com/karmakaragradwip02/rice_image_detection_cnn.mlflow",
            all_params=self.params
        )

        return model_evaluation_config

In [8]:
import json
import numpy as np
from pathlib import Path
from urllib.parse import urlparse
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, cohen_kappa_score

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def load_model(self):
        return tf.keras.models.load_model(self.config.trained_model_dir)
    
    def val_set(self):
        val_folder = self.config.val_dir
        val_datagen = ImageDataGenerator(rescale=1./255)
        val_set = val_datagen.flow_from_directory(
            val_folder,
            target_size=(64, 64),
            batch_size=32,
            class_mode='categorical',
            shuffle=False)
        return val_set
    
    def log_into_mlflow(self, model, val_set):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        # Read and parse the history data
        history_path = Path(self.config.history_dir)
        if history_path.is_file():
            with history_path.open('r') as f:
                history_data = json.load(f)
                
            # Log each epoch's metrics individually
            with mlflow.start_run():
                mlflow.log_params(self.config.all_params)
                y_pred = np.argmax(model.predict(val_set), axis=1)
                y_true = val_set.classes

                # Calculate precision and recall
                precision = precision_score(y_true, y_pred, average='macro')
                recall = recall_score(y_true, y_pred, average='macro')
                m_accuracy = accuracy_score(y_true, y_pred)
                f1 = f1_score(y_true, y_pred, average='macro')
                kappa = cohen_kappa_score(y_true, y_pred)
                
                # Log metrics
                mlflow.log_metric('Model Accuracy', m_accuracy)
                mlflow.log_metric('Model Precision', precision)
                mlflow.log_metric('Model Recall', recall)
                mlflow.log_metric('Model F1 Score', f1)
                mlflow.log_metric('Model Kappa', kappa)

                for epoch, (loss, accuracy, val_loss, val_accuracy) in enumerate(zip(
                        history_data["loss"],
                        history_data["accuracy"],
                        history_data["val_loss"],
                        history_data["val_accuracy"])):
                    mlflow.log_metric("loss", loss, step=epoch)
                    mlflow.log_metric("accuracy", accuracy, step=epoch)
                    mlflow.log_metric("val_loss", val_loss, step=epoch)
                    mlflow.log_metric("val_accuracy", val_accuracy, step=epoch)
                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(model, "model", registered_model_name="custom_model")
                else:
                    mlflow.keras.log_model(model, "model")

In [10]:
try:
    config = ConfigureationManager()
    eval_config = config.get_model_evaluation_config()
    evaluation = ModelEvaluation(eval_config)
    model = evaluation.load_model()
    val_set = evaluation.val_set()
    evaluation.log_into_mlflow(model, val_set)
except Exception as e:
   raise e

[2024-07-02 17:24:59,996: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-02 17:25:00,001: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-02 17:25:00,003: INFO: common: created directory at: artifacts]
[2024-07-02 17:25:00,004: INFO: common: created directory at: artifacts/evaluation]
Found 15000 images belonging to 5 classes.
469/469 [==============================] - 37s 77ms/step


2024/07/02 17:25:44 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-07-02 17:25:45,792: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _update_step_xla while saving (showing 3 of 3). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Karma\AppData\Local\Temp\tmpdae1w7n3\model\data\model\assets
[2024-07-02 17:25:46,426: INFO: builder_impl: Assets written to: C:\Users\Karma\AppData\Local\Temp\tmpdae1w7n3\model\data\model\assets]


c:\Users\Karma\anaconda3\envs\tf_env\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'custom_model' already exists. Creating a new version of this model...
2024/07/02 17:26:13 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: custom_model, version 4
Created version '4' of model 'custom_model'.
